In [0]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import torch.optim as optim

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
transform_train = transforms.Compose([
                                      transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # mean and std in 3 channels with mean and std of 0.5 an 0.5
])

transform_test = transforms.Compose([
                                     transforms.RandomResizedCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
trainset = torchvision.datasets.CIFAR10(root ='./data', train =True, download = True, transform = transform_train)
testset = torchvision.datasets.CIFAR10(root ='./data', train =False, download = True, transform= transform_test)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
num_classes =10 





In [0]:
# Pretrained Models.

# **Pretrained Models.**

In [0]:
batch_size = 16
trainloader = torch.utils.data.DataLoader(trainset, batch_size= batch_size, shuffle = True)
testloder = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

loss_fn = nn.CrossEntropyLoss()


In [0]:
def evaluation(dataloader, model):
  total, correct =0 , 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.do(device), labels.to(device)
    outputs = model(inputs)

    _, pred = torch.max(outputs, 1) # gives the plce whee the vlaue is highest.
    total += labels.size(0)
    correct += (pred == labels).sum().item()

  return 100* correct / total 

In [0]:
def pretrained_models(model):
  if model == "vgg":
    vgg = models.vgg16_bn(pretrained=True) # bn is batch Norm
    for param in vgg.parameters():
      param.requires_grad = False
    final_in_features = vgg.classifier[6].in_features
    vgg.classifier[6] = nn.Linear(final_in_features, num_classes)
    vgg = vgg.to(device)
    train(vgg)

  if model == "resnet":
    resnet = models.resnet18(pretrained=True) 
    for param in resnet.parameters():
      param.requires_grad = False
    in_features = resnet.fc.in_features
    resnet.fc = nn.Linear(in_features, num_classes)
    resnet = resnet.to(device)
    train(resnet)

  if model == "inception":
    inception = models.inception_v3(pretrained = True)
    for param in inception.parameters():
      param.requires_grad = False
    aux_in_features = inception.AuxLogits.fc.in_features
    inception.AuxLogits.fc = nn.Linear(aux_in_features, num_classes)
    in_features= inception.fc.in_features
    inception.fc = nn.Linear(in_features, num_classes)
    transform_train = transforms.Compose([
                                      transforms.RandomResizedCrop(299),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  ])
    transform_test = transforms.Compose([
                                     transforms.RandomResizedCrop(299),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])
    trainset = torchvision.datasets.CIFAR10(root ='./data', train =True, download = True, transform = transform_train)
    testset = torchvision.datasets.CIFAR10(root ='./data', train =False, download = True, transform= transform_test)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size= batch_size, shuffle = True)
    testloder = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)
    train_inception(inception)
    






In [0]:
def evaluation_inception(dataloader, model):
  total, correct =0 , 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.do(device), labels.to(device)
    outputs, aux_outputs = model(inputs)

    _, pred = torch.max(outputs, 1) # gives the plce whee the vlaue is highest.
    total += labels.size(0)
    correct += (pred == labels).sum().item()

  return 100* correct / total 

In [0]:
def train_inception(model):
  loss_epochs_arr = []
  max_epochs = 4
  batch_size =16
  min_loss = 1000
  opt = optim.SGD(inception.parameters(), lr = 0.01)
  loss_fn = nn.CrossEntropyLoss()

  o

  for epoch in range(max_epochs):
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      opt.zero_grad()

      outputs, aux_ouputs = model(inputs)

      loss = loss_fn(outputs, labels) + 0.3 * loss_fn(aux_ouputs, labels)

      loss.backward()

      opt.step()

      if min_loss > loss.item():
        min_loss = loss.item()
        best_model = copy.deepcopy(resnet.state_dict())
        print('Min Loss %0.2f'% min_loss)

      if i % 100 == 0:
        print("Iterations %d/%d loss = %0.2f"%(i, n_iters, loss.item()))

      del inputs, labels, outputs
      torch.cuda.empty_cache()


    loss_epochs_arr.append(loss)

    print('Epoch: %d/%d , test acc : %0.2f, train Acc %0.2f'% (epoch, max_epochs, evaluation(testloader, resnet), evaluation(trainloader, resnet)))


  plt.plot(loss_epoch_arr)
  plt.show()

  

In [0]:
def train(model):
  loss_epochs_arr = []
  max_epochs = 1
  batch_size = 16
  n_iters = np.ceil(50000/ batch_size)
  opt = optim.SGD(model.parameters(), lr = 0.05)

  for epoch in range(max_epochs):
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)
      opt.zero_grad()
      outputs = model(inputs)
      loss = loss_fn(outputs, labels)
      loss.backward()
      opt.step()

      del inputs, labels
      torch.cuda.empty_cache()

      if i%100 == 0:
        print("Iterations %d/%d loss = %0.2f"%(i, n_iters, loss.item()))

    loss_epochs_arr.append(loss)
    print('Epoch: %d/%d , test acc : %0.2f, train Acc %0.2f'% (epoch, max_epochs, evaluation(testloader, model), evaluation(trainloader, model)))

# VGG-16

In [0]:
pretrained_models("vgg")

Iterations 0/3125 loss = 2.48


KeyboardInterrupt: ignored

## Resnet 

In [0]:
pretrained_models("resnet")

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth



Iterations 0/3125 loss = 2.49
Iterations 100/3125 loss = 1.79
Iterations 200/3125 loss = 2.80
Iterations 300/3125 loss = 1.15


KeyboardInterrupt: ignored

## Inception_v3

In [24]:
pretrained_models("inception")

KeyboardInterrupt: ignored

In [29]:
squeezenet = models.squeezenet1_0(pretrained=True)

Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to /root/.cache/torch/checkpoints/squeezenet1_0-a815701f.pth


In [30]:
squeezenet

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   